In [ ]:
           # --- Global mean per Fourier term ---
            mu_season = pm.Normal(
                "mu_season",
                mu=0,
                sigma=sigma_vec,
                dims="season_term"
            ) # Global seasonal term effects (6,)
        
            # --- Market-level deviations ---
            sigma_market = pm.HalfNormal("sigma_season_market", sigma=market_sigma, dims = "season_term") # Market deviations from global for seasonal terms 
            z_market = pm.Normal("z_season_market", mu=0, sigma=1, dims=("market", "season_term")) # Non-centered directional prior
        
            # --- Product-level deviations (within market) ---
            sigma_product = pm.HalfNormal("sigma_season_product", sigma=product_sigma, dims = "season_term") # Product deviation from global for seasonal terms
            z_product = pm.Normal("z_season_product", mu=0, sigma=1, dims=("product", "season_term")) # Non-centered directional prior

            # --- Product x Market specific deviations (products seasonality differing per market) --- 
            sigma_product_market = pm.HalfNormal('sigma_market_product', sigma = market_product_sigma, dims = "season_term") # Market x product specific deviations from global
            z_product_market = pm.Normal("z_season_product_market", mu=0, sigma=1, dims=("market", "product", "season_term")) # Non-centered directional prior

            # Defining market x product specific coefficients
            beta_season = pm.Deterministic(
                "beta_season",
                mu_season[None, None, :] + 
                sigma_market * z_market[:,None,:] + 
                sigma_product * z_product[None, :, :] + 
                sigma_product_market * z_product_market,
                dims=("market", "product", "season_term")
            )
        
            # --- Combine Fourier basis with coefficients ---
            season_term = at.sum(
                X_season * beta_season[market_idx, prod_idx, :],
                axis=1
            )
        
        else:
            season_term = at.zeros_like(trend)  # or at.constant(0.0)

In [ ]:
beta_mpc = mu_global[None, None, :] + sigma_market * z_market[:, None, :] + sigma_product * z_product[None,:,:] + sigma_prod_mark * z_prod_mark, dims = ("market", "product", "channel")

media_contri_ch = pm.determinstic("media_contri_ch", beta_mpc[market_idx, product_idx, :] * X_media, dims = ("obs", "channel")